In [12]:
from typing import List, Dict, Optional, Any
def _as_dict(obj: Any) -> dict:
    if isinstance(obj, dict):
        return obj
    # objetos do SDK podem ter .model_dump() / .to_dict(), tentamos extrair
    for attr in ("model_dump", "to_dict", "dict"):
        fn = getattr(obj, attr, None)
        if callable(fn):
            try:
                return fn()
            except Exception:
                pass
    # fallback: tentar attrs conhecidos manualmente
    d = {}
    for k in ("type", "text", "content", "value"):
        if hasattr(obj, k):
            d[k] = getattr(obj, k)
    return d

def _extract_text(resp: Any) -> str:
    # 1) caminho feliz (Responses API)
    text = getattr(resp, "output_text", None)
    if isinstance(text, str) and text.strip():
        return text.strip()

    chunks: List[str] = []

    # 2) Responses API — variantes (cobre 'text', 'output_text' e também 'reasoning')
    outputs = getattr(resp, "output", None) or getattr(resp, "outputs", None) or []
    try:
        for item in outputs:
            item_d = _as_dict(item)
            content = item_d.get("content", []) or []
            for block in content:
                b = _as_dict(block)
                btype = b.get("type")
                if btype in ("output_text", "text", "reasoning"):
                    t = b.get("text")
                    if isinstance(t, str) and t.strip():
                        chunks.append(t.strip())
                    else:
                        td = _as_dict(t)
                        val = td.get("value") or td.get("text")
                        if isinstance(val, str) and val.strip():
                            chunks.append(val.strip())
                if btype == "message":
                    inner = b.get("content", []) or []
                    for sb in inner:
                        sb_d = _as_dict(sb)
                        if sb_d.get("type") in ("output_text", "text", "reasoning"):
                            tt = sb_d.get("text")
                            if isinstance(tt, str) and tt.strip():
                                chunks.append(tt.strip())
                            else:
                                ttd = _as_dict(tt)
                                sval = ttd.get("value") or ttd.get("text")
                                if isinstance(sval, str) and sval.strip():
                                    chunks.append(sval.strip())
    except Exception:
        pass

In [8]:
# Teste de sanidade isolado (rode em um script separado) — deve imprimir "ok"
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
resp = client.responses.create(
    model="gpt-5-mini",
    input="Olá",
    max_output_tokens=256,
)
print("Saída:", repr(resp.output_text))  # Esperado: 'ok'

Saída: 'Olá! Como posso ajudar você hoje?'


In [ ]:
import openai, sys; st.caption(f"openai={openai.version} python={sys.version}")

NameError: name 'st' is not defined